In [ ]:
import IPython.display as ipd

!pip install face_alignment

ipd.clear_output()

In [ ]:
from glob import glob
import shutil
from pathlib import Path
import random

import numpy as np
from PIL import Image
import imageio
import cv2
from tqdm.auto import tqdm

from google.colab import drive
drive.mount('/content/gdrive')

ROOT_DIR = '/content/gdrive/MyDrive/기계학습프로젝트/12차시'
IMG_DIR = f'{ROOT_DIR}/images'
CROP_IMG_DIR = f'{ROOT_DIR}/images_crop'

device = 'cuda:0'

Mounted at /content/gdrive


In [ ]:
!nvidia-smi

In [ ]:
from face_alignment.detection.sfd import FaceDetector

fd = FaceDetector(device)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class RectDataset(Dataset):
    def __init__(self, root_dir, rect_size = 512):
        self.images = sorted(glob(f'{root_dir}/*/*.jpg'))
        self.rect_size = rect_size

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        path = self.images[idx]

        im = imageio.imread(path)
        im = RectDataset.to_rect_img(im, self.rect_size)
        im = im.transpose(2, 0, 1).astype(np.float32)

        pid = '/'.join(path.split('/')[-2:]) 
        return im, pid

    # 종횡비를 변경하지 않고 리사이즈 한 후 패딩을 해서 정사각 이미지로 만든다 
    def to_rect_img(im, rect_size):
        rs = rect_size
        h, w, _ = im.shape
        s = rs/max(h,w)
        im = cv2.resize(im, (int(w*s), int(h*s)), cv2.INTER_CUBIC)
        h, w, _ = im.shape
        return np.pad(im, 
                    (((rs - h)//2, rs - h - (rs - h)//2), 
                    ((rs - w)//2, rs - w - (rs - w)//2),
                    (0,0)), 
                    mode='constant', constant_values=128)
            
                

ds = RectDataset(IMG_DIR)
dl = DataLoader(ds, batch_size=32, num_workers=2)

len(ds), len(dl)

In [ ]:
for ims, labels in tqdm(dl):

    ims = ims.to(device)
    boxes_batch = fd.detect_from_batch(ims)

    top_prob = lambda boxes: sorted(boxes, key=lambda e: e[4], reverse=True)[:1]
    box_batch = [top_prob(e) for e in boxes_batch]

    to_box = lambda boxes: [(e[:4].astype(np.int32), e[4])for e in boxes]
    box_batch = [to_box(e) for e in box_batch]

    for box, im, label in zip(box_batch, ims, labels):
        if not box:
            continue
        (x1, y1, x2, y2), _  = box[0]
        crop = im.cpu().numpy().transpose(1,2,0)[y1:y2, x1:x2, :].astype(np.uint8)
        crop = RectDataset.to_rect_img(crop, 112)
        save_path = f'{CROP_IMG_DIR}/{label}'
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)
        imageio.imwrite(save_path, crop)

In [ ]:
drive.flush_and_unmount()